# Logistic regression

### Design matrix
Start with one predictor: stimulus

In [1]:
"""
IMPORTS
"""
import numpy as np
import numpy.random as npr
import os
import random
import pandas as pd
import matplotlib.pyplot as plt

# --Get my functions
functions_path = "../../Functions/"
os.chdir(functions_path)
from one_functions_generic import get_trials, performance_metrics, bin_frac, prepro

npr.seed(65)

In [2]:
"""
Load data
"""
trials_dir = '/home/ines/repositories/representation_learning_variability/DATA/'
#trials_dir = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/'
all_trials = pd.read_csv(trials_dir + "proficient_one.csv")  

From all data, focus on an example session of an example mouse 
Total of 530 trials should be enough for a sigmoid fit

In [3]:
mouse_example = all_trials.loc[(all_trials['subject_nickname']=='KS014') & (all_trials['session']=='94535612-5064-40aa-aa05-8c08c2fa618a')]

In [4]:
mouse_example = prepro(mouse_example)

/home/ines/repositories/representation_learning_variability/Functions/one_functions_generic.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trials['contrastLeft'] = trials['contrastLeft'].fillna(0)
/home/ines/repositories/representation_learning_variability/Functions/one_functions_generic.py:481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trials['contrastRight'] = trials['contrastRight'].fillna(0)
/home/ines/repositories/representation_learning_variability/Functions/one_functions_generic.py:482: Se

Functions

In [30]:
def logistic(x, theta):
    # Logistic function (sigmoid)
    y = 1/(np.exp(-x * theta)+1)
    return y


def initialize(dim):
    # Initialize theta array and bias 
    b=random.random()
    theta=np.random.rand(dim)
    return theta


def loglikelihood(y, x, theta):
    ll = np.sum(y * np.log(logistic(x, theta)) + (1-y)* np.log(1-logistic(x, theta)))
    return ll


def gradient(x, y, theta_vector):
    # Computed gradient for Bernoulli distribution
    sigma = logistic(x, theta_vector)
    n = len(y)
    grad = (np.sum(sigma - y) * x )/ n
    return grad
    
    
def gradient_descent(theta_dim, alpha, x, y, n_iter=50, tolerance=1e-06):
    # the algorithm will simultaneously update the theta values 
    # after each model fits to find the global minimum of our cost function.
    theta_vector = initialize(theta_dim)
    for _ in range(n_iter):
        diff = -alpha * gradient(x, y, theta_vector)
        if np.all(np.abs(diff) <= tolerance):
            break
        theta_vector += diff
    return theta_vector

In [31]:
x = np.array(mouse_example['signed_contrast'])
y = np.array(mouse_example['choice'])

In [32]:
theta_vector = initialize(1)
theta_vector

array([0.50571517])

In [33]:
gradient(x, y, theta_vector)

array([-0.82281406,  0.05142588, -0.20570352,  0.        ,  0.        ,
       -0.20570352,  0.20570352, -0.82281406, -0.10285176, -0.20570352,
       -0.10285176, -0.10285176,  0.        , -0.20570352, -0.10285176,
       -0.05142588, -0.05142588,  0.        ,  0.        ,  0.10285176,
       -0.05142588,  0.        ,  0.        , -0.82281406,  0.        ,
        0.82281406,  0.        , -0.10285176, -0.82281406,  0.10285176,
       -0.20570352, -0.82281406,  0.05142588, -0.10285176,  0.        ,
        0.05142588, -0.82281406, -0.10285176,  0.        ,  0.05142588,
        0.        ,  0.10285176,  0.05142588,  0.10285176,  0.10285176,
        0.        ,  0.        ,  0.82281406,  0.05142588,  0.        ,
        0.82281406,  0.10285176,  0.        , -0.20570352,  0.82281406,
        0.        , -0.20570352,  0.82281406, -0.82281406, -0.82281406,
        0.        ,  0.82281406,  0.10285176,  0.10285176,  0.20570352,
        0.82281406,  0.10285176,  0.        ,  0.        ,  0.05

In [34]:
theta_dim = 1
alpha = 0.01
theta_values = gradient_descent(theta_dim, alpha, x, y, n_iter=100, tolerance=1e-06)

ValueError: non-broadcastable output operand with shape (1,) doesn't match the broadcast shape (530,)

In [36]:
theta_values

array([-719.76859509])

In [42]:
plt.plot(x, y)

NameError: name 'plt' is not defined